# A Guide to Migrating Primitives from V1 to V2

## Introduction

The Qiskit Runtime V2 Primitives represent a major shift in how we execute quantum jobs. This notebook provides a **complete, runnable migration example**.

We will:
1.  **Start with a V1 Example**: Define and run a job using the legacy `Sampler` (V1) interface.
2.  **Walk Through Migration**: Step-by-step transformation of the code.
3.  **Finish with a V2 Example**: Run the same logic using the modern `SamplerV2` and compare the results.

In [1]:
# Universal Setup: Define the Circuit and Parameters
# This setup is common to both V1 and V2 examples.

import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit_ibm_runtime.fake_provider import FakeManilaV2

# 1. Initialize a Backend (Simulator)
backend = FakeManilaV2()

# 2. Create a Parametrized Circuit
theta = Parameter('theta')
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.rx(theta, 0)
qc.measure_all()

# 3. Define Parameter Values
# We will execute the circuit with 3 different values for theta
parameter_values = [0.0, np.pi/2, np.pi]

print("Setup Complete: Circuit and Parameters ready.")

Setup Complete: Circuit and Parameters ready.


## Step 1: The "Old" Way (V1 Interface)

In V1, we typically passed a list of circuits and a separate list of parameter values. The result was given as a "Quasi-Distribution".

> **Note**: We use the `Sampler` from `qiskit_ibm_runtime` (Legacy) here. It is deprecated but shown for educational comparison.

In [2]:
from qiskit_ibm_runtime import Sampler as SamplerV1
from qiskit.transpiler import generate_preset_pass_manager

# 1. Legacy Transpilation (Optional in V1, but good practice)
# V1 was more forgiving with non-ISA circuits, but we transpile anyway.
pm = generate_preset_pass_manager(target=backend.target, optimization_level=1)
v1_circuit = pm.run(qc)

# 2. Initialize V1 Sampler
sampler_v1 = SamplerV1(backend=backend)

# 3. Run V1 Job
# Notice the separate list for parameter_values. This was often a source of confusion (broadcasting ambiguity).
print("Running V1 Job...")
job_v1 = sampler_v1.run(circuits=[v1_circuit]*3, parameter_values=parameter_values)
result_v1 = job_v1.result()

# 4. Show V1 Results (Quasi-Distributions)
print("\n--- V1 Results (Quasi-Dists) ---")
for i, dist in enumerate(result_v1.quasi_dists):
    print(f"Theta {parameter_values[i]:.2f}: {dist}")

TypeError: SamplerV2.__init__() got an unexpected keyword argument 'backend'

## Step 2: The Migration Walkthrough

To move to V2, we need to apply three major changes:

### Change 1: The Import
*   **V1**: `from qiskit_ibm_runtime import Sampler`
*   **V2**: `from qiskit_ibm_runtime import SamplerV2`

### Change 2: The Input (Pubs)
*   **V1**: `run(circuits=[c1, c2], parameter_values=[p1, p2])`
*   **V2**: `run([(c1, p1), (c2, p2)])`
    *   We must group the circuit and its parameters into a tuple called a **Pub**.

### Change 3: The Output
*   **V1**: `result.quasi_dists` (Normalized probabilities)
*   **V2**: `result[0].data.meas.get_counts()` (Raw counts or bitstrings)
    *   V2 results are accessed per-pub and per-register.

## Step 3: The "New" Way (V2 Interface)

Now let's apply these changes to run the exact same experiment.

In [ ]:
from qiskit_ibm_runtime import SamplerV2

# 1. Strict Transpilation (Required for V2)
# V2 requires circuits to comply with the backend's ISA (Instruction Set Architecture).
isa_circuit = pm.run(qc)

# 2. Initialize V2 Sampler
sampler_v2 = SamplerV2(mode=backend)

# 3. Run V2 Job using Pubs
# We create a SINGLE pub that contains the circuit and ALL parameter sets.
# Pub Format: (circuit, parameter_values)
pub = (isa_circuit, parameter_values)

print("Running V2 Job...")
job_v2 = sampler_v2.run([pub])
result_v2 = job_v2.result()

# 4. Show V2 Results (Bitstrings/Counts)
# Result access is hierarchical: result -> pub_result -> data -> register -> format
print("\n--- V2 Results (Counts) ---")
pub_result = result_v2[0]
counts_list = pub_result.data.meas.get_counts()

for i, counts in enumerate(counts_list):
    print(f"Theta {parameter_values[i]:.2f}: {counts}")

## Step 4: Comparison & Conclusion

Let's compare what we got:

*   **V1 Output**: `{'3': 0.9, '0': 0.1}` (Probabilities, key is decimal integer)
*   **V2 Output**: `{'11': 900, '00': 100}` (Counts, key is binary string)

**Summary of Benefits**:
1.  **Clarity**: The "Pub" structure `(circuit, params)` makes it obvious which parameters belong to the circuit.
2.  **Raw Data**: V2 gives you the actual counts `900`, not just a probability `0.9`. This preserves statistical information.
3.  **Typos**: V2 uses `meas` (register name) to access data, allowing support for circuits with multiple registers (`meas1`, `meas2`).